## Introduction
This notebook is designed to guide you through the process of building a simple retrieval augmented generation application using Langchain, ChromaDB and OpenAI's GPT-4 LLM. 

![](../assets/4-RAG.png)

### RAG: Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) combines the power of language models with external content retrieval to answer questions based on specific content. This approach allows for more accurate and contextually relevant responses by leveraging a database of information.

The process involves several key steps:
- **Content Retrieval and Storage**: First, we ingest and store our content in a searchable format. This involves fetching content from a URL, splitting it into manageable parts, embedding these parts for semantic search, and storing them in a Vector DB.
- **Question Answering**: When a user question is posed, the system retrieves relevant content from the Vector DB and uses it to generate an answer with the LLM.

This notebook will walk you through setting up a RAG system using ChromaDB for content storage and retrieval, and OpenAI's GPT-4 for question answering.

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

False

# Run basic example from Langchain Vectorstores Chroma page

[Read more here](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)

# Basic Example

In [2]:
import rich
from goob_ai import debugger

In [3]:
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("example_data/state_of_the_union.txt")
documents = loader.load()


In [4]:
rich.inspect(loader)

rich.inspect(documents)

╭───────── <class 'langchain_community.document_loaders.text.TextLoader'> ─────────╮
│ Load text file.                                                                  │
│                                                                                  │
│ ╭──────────────────────────────────────────────────────────────────────────────╮ │
│ │ <langchain_community.document_loaders.text.TextLoader object at 0x105087430> │ │
│ ╰──────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                  │
│ autodetect_encoding = False                                                      │
│            encoding = None                                                       │
│           file_path = 'example_data/state_of_the_union.txt'                      │
╰──────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── <class 'list'> ────────────────────────────────────────╮
│ Built-in mutable sequence.                                                                     │
│                                                                                                │
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                          │ │
│ │ │   Document(                                                                              │ │
│ │ │   │   page_content='Madam Speaker, Madam Vice President, our First Lady and Seco'+38479, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                         │ │
│ │ │   )                                                                                      │ │
│ │ ]                                                                                          │ │
│ ╰────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                │
│ 36 attribute(s) not shown. Run inspect(inspect) for options.                                   │
╰────────────────────────────────────────────────────────────────────────────────────────────────╯

In [5]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [6]:
rich.inspect(text_splitter)

rich.inspect(docs)

╭───────── <class 'langchain_text_splitters.character.CharacterTextSplitter'> ─────────╮
│ Splitting text that looks at characters.                                             │
│                                                                                      │
│ ╭──────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <langchain_text_splitters.character.CharacterTextSplitter object at 0x11f0f6bc0> │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                      │
│ 40 attribute(s) not shown. Run inspect(inspect) for options.                         │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────── <class 'list'> ───────────────────────────────────────╮
│ Built-in mutable sequence.                                                                   │
│                                                                                              │
│ ╭──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                        │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='Madam Speaker, Madam Vice President, our First Lady and Seco'+879, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='Groups of citizens blocking tanks with their bodies. Everyon'+935, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='Putin’s latest attack on Ukraine was premeditated and unprov'+899, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='We are inflicting pain on Russia and supporting the people o'+887, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='And tonight I am announcing that we will join our allies in '+916, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='For that purpose we’ve mobilized American ground forces, air'+891, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='And I’m taking robust action to make sure the pain of our sa'+894, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='We see the unity among leaders of nations and a more unified'+922, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(                                                                            │ │
│ │ │   │   page_content='And so many families are living paycheck to paycheck, strugg'+833, │ │
│ │ │   │   metadata={'source': 'example_data/state_of_the_union.txt'}                       │ │
│ │ │   ),                                                                                   │ │
│ │ │   Document(      

In [7]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/Users/malcolm/dev/bossjones/goob_ai/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/malcolm/dev/bossjones/goob_ai/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/malcolm/dev/bossjones/goob_ai/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.

In [8]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# Persistant saving to disk example

In [9]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# passing a Chroma client into langchain

In [10]:
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embedding_function,
)

print("There are", langchain_chroma._collection.count(), "in the collection")

Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3


There are 3 in the collection


# Use Chroma in a docker container


In [11]:
# create the chroma client
import uuid

import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(host="localhost", port="8010", settings=Settings(allow_reset=True))
client.reset()  # resets the database
collection = client.create_collection("my_collection")
for doc in docs:
    collection.add(
        ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
    )

# tell LangChain to use our client and collection name
db4 = Chroma(
    client=client,
    collection_name="my_collection",
    embedding_function=embedding_function,
)
query = "What did the president say about Ketanji Brown Jackson"
docs = db4.similarity_search(query)
print(docs[0].page_content)

Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context le

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# Chromadb: Update and Delete

While building toward a real application, you want to go beyond adding data, and also update and delete data.

Chroma has users provide ids to simplify the bookkeeping here. ids can be the name of the file, or a combined has like filename_paragraphNumber, etc.

Chroma supports all these operations - though some of them are still being integrated all the way through the LangChain interface. Additional workflow improvements will be added soon.

Here is a basic example showing how to do various operations:

In [15]:
# create simple ids
ids = [str(i) for i in range(1, len(docs) + 1)]

# add data
example_db = Chroma.from_documents(docs, embedding_function, ids=ids)
docs = example_db.similarity_search(query)
print(docs[0].metadata)

# update the metadata for a document
docs[0].metadata = {
    "source": "example_data/state_of_the_union.txt",
    "new_value": "hello world",
}
example_db.update_document(ids[0], docs[0])
print(example_db._collection.get(ids=[ids[0]]))

# delete the last document
print("count before", example_db._collection.count())
example_db._collection.delete(ids=[ids[-1]])
print("count after", example_db._collection.count())

{'source': 'example_data/state_of_the_union.txt'}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'new_value': 'hello world', 'source': 'example_data/state_of_the_union.txt'}], 'documents': ['Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'], 'uris': N

# Use OpenAI Embeddings


In [ ]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings()
new_client = chromadb.EphemeralClient()
openai_lc_client = Chroma.from_documents(
    docs, embeddings, client=new_client, collection_name="openai_collection"
)

query = "What did the president say about Ketanji Brown Jackson"
docs = openai_lc_client.similarity_search(query)
print(docs[0].page_content)

# Use OpenAI Embeddings w/ docker chroma

In [30]:
import os
from goob_ai.aio_settings import AioSettings, aiosettings

os.environ["FAKE"] = f"{aiosettings.openai_api_key.get_secret_value()}"
os.environ["FAKE"]


# rich.print(aiosettings)

''

In [25]:
# create the chroma client
import uuid

import os

import chromadb
from chromadb.config import Settings
from langchain_openai import OpenAIEmbeddings
from getpass import getpass

OPENAI_API_KEY = getpass()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [31]:

embeddings = OpenAIEmbeddings()

client = chromadb.HttpClient(host="localhost", port="8010", settings=Settings(allow_reset=True))
# client.reset()  # resets the database
collection = client.create_collection("openai_collection")

openai_lc_client = Chroma.from_documents(
    docs, embeddings, client=client, collection_name="openai_collection"
)

query = "What did the president say about Ketanji Brown Jackson"
docs = openai_lc_client.similarity_search(query)
print(docs[0].page_content)

# # for doc in docs:
# #     collection.add(
# #         ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
# #     )

# # # tell LangChain to use our client and collection name
# # db4 = Chroma(
# #     client=client,
# #     collection_name="my_collection",
# #     embedding_function=embedding_function,
# # )
# # query = "What did the president say about Ketanji Brown Jackson"
# # docs = db4.similarity_search(query)
# # print(docs[0].page_content)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# Other Information: Similarity search with score

In [33]:
# The returned distance score is cosine distance. Therefore, a lower score is better.
docs = db.similarity_search_with_score(query)
docs[0]

(Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': 'example_data/state_of_the_union.txt'}),
 1.1972053050994873)

In [12]:
# # NOTE: This is from GP, let's play with this later
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_openai import AzureOpenAIEmbeddings
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_chroma import Chroma
# import bs4

# # Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# # Split the content into manageable chunks for better retrieval.
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

# # Embed the chunks and store them in ChromaDB for efficient retrieval.
# vectorstore = Chroma.from_documents(documents=splits, embedding=AzureOpenAIEmbeddings(azure_deployment=os.environ["AZURE_EMBEDDINGS_DEPLOYMENT"]))

In [13]:
# import os
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import AzureChatOpenAI
# from langchain_core.output_parsers import StrOutputParser

# # Set up the RAG chain for retrieving and generating answers.
# retriever = vectorstore.as_retriever()
# system_prompt = ("""
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
# Context: {context}
# """)

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{question}"),
#     ]
# )


# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)


# # Initialize the model with our deployment of Azure OpenAI
# model = AzureChatOpenAI(azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"])

# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | model
#     | StrOutputParser()
# )

In [14]:
# question = "What is tool usage?"
# rag_chain.invoke(question)